In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from Bio import Entrez, SeqIO

In [2]:
sacc_id = 'R64'
yl_id = 'W29'
ids = [sacc_id, yl_id]


### Generate faa and fna files

In [6]:
def gbff_to_fna_faa(gbff_file, fna_outfile, faa_outfile):
    
    with open(fna_outfile, "w") as fna_out, open(faa_outfile, "w") as faa_out:
        for record in SeqIO.parse(gbff_file, "genbank"):
            # Write full genome/contig sequence to .fna
            fna_out.write(f">{record.id} {record.description}\n{record.seq}\n")

            # Write all protein translations to .faa
            for feature in record.features:
                if feature.type == "CDS" and "translation" in feature.qualifiers:
                    protein_seq = feature.qualifiers["translation"][0]
                    locus_tag = feature.qualifiers.get("locus_tag", ["unknown"])[0]
                    product = feature.qualifiers.get("product", [""])[0]
                    header = f">{locus_tag} {product}".strip()
                    faa_out.write(f"{header}\n{protein_seq}\n")
                    
                    

In [7]:
gbff_to_fna_faa("genomes/" +sacc_id+".gbff", "nucl/" +sacc_id+".fna", "prot/" +sacc_id+".faa")
gbff_to_fna_faa("genomes/" +yl_id+".gbff", "nucl/" +yl_id+".fna", "prot/" +yl_id+".faa")


In [8]:
# Define a function to make blast database for either protein of nucleotide
def make_blast_db(id,folder='prot',db_type='prot'):
    import os
    out_file ='%s/%s.fa.pin'%(folder, id)
    files =glob('%s/*.fa.pin'%folder)
    
    if out_file in files:
        print (id, 'already has a blast db')
        return
    if db_type=='nucl':
        ext='fna'
    else:
        ext='faa'

    cmd_line='makeblastdb -in %s/%s.%s -dbtype %s' %(folder, id, ext, db_type)
    
    print ('making blast db with following command line...')
    print (cmd_line)
    os.system(cmd_line)

In [10]:
for seq in ids:
    new_seq = seq
    make_blast_db(new_seq,folder='prot',db_type='prot')

making blast db with following command line...
makeblastdb -in prot/R64.faa -dbtype prot


Building a new DB, current time: 06/19/2025 12:07:40
New DB name:   /home/kkrishnan/SBRG/imodulons/yarrowia_lipolytica_2025/data/blast/prot/R64.faa
New DB title:  prot/R64.faa
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 6016 sequences in 0.131303 seconds.
making blast db with following command line...
makeblastdb -in prot/W29.faa -dbtype prot


Building a new DB, current time: 06/19/2025 12:07:40
New DB name:   /home/kkrishnan/SBRG/imodulons/yarrowia_lipolytica_2025/data/blast/prot/W29.faa
New DB title:  prot/W29.faa
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 7918 sequences in 0.136089 seconds.


In [11]:
# define a function to run BLASTp
def run_blastp(seq,db,in_folder='prot', out_folder='bbh', out=None,outfmt=6,evalue=0.001,threads=1):
    import os
    if out==None:
        out='%s/%s_vs_%s.txt'%(out_folder, seq, db)
        print(out)
    
    files =glob('%s/*.txt'%out_folder)
#    if out in files:
#        print (seq, 'already blasted')
#        return
    
    print ('blasting %s vs %s'%(seq, db))
    
    db = '%s/%s.faa'%(in_folder, db)
    seq = '%s/%s.faa'%(in_folder, seq)
    cmd_line='blastp -db %s -query %s -out %s -evalue %s -outfmt %s -num_threads %i' \
    %(db, seq, out, evalue, outfmt, threads)
    
    print ('running blastp with following command line...')
    print (cmd_line)
    os.system(cmd_line)
    return out

In [12]:
# define a function to get sequence length 

def get_gene_lens(query, in_folder='prot'):
    file = '/home/kkrishnan/SBRG/Sequencing/Y-Lipolytica/blast/prot/%s/%s.faa'%(in_folder, query)
    file = '%s/%s.faa'%(in_folder, query)
    handle = open(file)
    records = SeqIO.parse(handle, "fasta")
    out = []
    
    for record in records:
        out.append({'gene':record.name, 'gene_length':len(record.seq)})
    
    out = pd.DataFrame(out)
    return out

In [13]:
# define a function to get Bi-Directional BLASTp Best Hits
def get_bbh(query, subject, in_folder='bbh'):    
    
    #Utilize the defined protein BLAST function
    run_blastp(query, subject)
    run_blastp(subject, query)
    
    query_lengths = get_gene_lens(query, in_folder='prot')
    subject_lengths = get_gene_lens(subject, in_folder='prot')
    
    #Define the output file of this BLAST
    out_file = '%s/%s_vs_%s_parsed.csv'%(in_folder,query, subject)
    files=glob('%s/*_parsed.csv'%in_folder)
    
    #Combine the results of the protein BLAST into a dataframe
    print ('parsing BBHs for', query, subject)
    cols = ['gene', 'subject', 'PID', 'alnLength', 'mismatchCount', 'gapOpenCount', 'queryStart', 'queryEnd', 'subjectStart', 'subjectEnd', 'eVal', 'bitScore']
    bbh=pd.read_csv('%s/%s_vs_%s.txt'%(in_folder,query, subject), sep='\t', names=cols)
    bbh = pd.merge(bbh, query_lengths) 
    bbh['COV'] = bbh['alnLength']/bbh['gene_length']
    
    bbh2=pd.read_csv('%s/%s_vs_%s.txt'%(in_folder,subject, query), sep='\t', names=cols)
    bbh2 = pd.merge(bbh2, subject_lengths) 
    bbh2['COV'] = bbh2['alnLength']/bbh2['gene_length']
    out = pd.DataFrame()
    
    # Filter the genes based on coverage
    bbh = bbh[bbh.COV>=0.25]
    bbh2 = bbh2[bbh2.COV>=0.25]
    
    #Delineate the best hits from the BLAST
    for g in bbh.gene.unique():
        res = bbh[bbh.gene==g]
        if len(res)==0:
            continue
        best_hit = res.loc[res.PID.idxmax()]
        best_gene = best_hit.subject
        res2 = bbh2[bbh2.gene==best_gene]
        if len(res2)==0:
            continue
        best_hit2 = res2.loc[res2.PID.idxmax()]
        best_gene2 = best_hit2.subject
        if g==best_gene2:
            best_hit['BBH'] = '<=>'
        else:
            best_hit['BBH'] = '->'
        out=pd.concat([out, pd.DataFrame(best_hit).transpose()])
    
    #Save the final file to a designated CSV file    
    out.to_csv(out_file)

In [14]:
get_bbh('R64','W29', in_folder='bbh')

bbh/R64_vs_W29.txt
blasting R64 vs W29
running blastp with following command line...
blastp -db prot/W29.faa -query prot/R64.faa -out bbh/R64_vs_W29.txt -evalue 0.001 -outfmt 6 -num_threads 1
bbh/W29_vs_R64.txt
blasting W29 vs R64
running blastp with following command line...
blastp -db prot/R64.faa -query prot/W29.faa -out bbh/W29_vs_R64.txt -evalue 0.001 -outfmt 6 -num_threads 1
parsing BBHs for R64 W29


/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'
/tmp/ipykernel_191074/929511565.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

...

/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'
/tmp/ipykernel_191074/929511565.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

/tmp/ipykernel_191074/929511565.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_191074/929511565.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'
/tmp/ipykernel_191074/929511565.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'
/tmp/ipykernel_191074/929511565.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [36]:
'''
Generate gene table
'''

from pymodulon.gene_util import *

sacc_gene_table = gff2pandas('../Sequences/R64.gff',index='locus_tag')
sacc_gene_table.index.name = 'gene'
sacc_gene_table = sacc_gene_table.rename(columns={'gene_name':'sacc_gene_name','gene_product':'sacc_gene_product'})

bbh_results = pd.read_csv('bbh/R64_vs_W29_parsed.csv',index_col='Unnamed: 0')
bbh_results = bbh_results.merge(sacc_gene_table, on='gene', how='left')
bbh_results = bbh_results.sort_values('PID', ascending=False).drop_duplicates(subset='subject', keep='first').reset_index()

In [37]:
bbh_results.to_csv('bbh/homologue_maps.csv')